In [4]:
import numpy as np
import cv2 as cv

In [5]:
class Cartoonizer:
    def __init__(self):
        pass
    
    def render(self, img_rgb):
        # read the image
        img_rgb = cv.imread(img_rgb)
        img_rgb = cv.resize(img_rgb, (1366, 768))
        numDownSamples = 2 # number of downscaling steps
        numBilateralFilters = 50 # number of bilateral filtering steps
        
        # downsample image using Gaussian pyramid
        # pyramid or pyramid representation is a type of multi-scale signal representation in which a signal or an image
        # is subject to repeated smoothing and subsampling.
        img_color = img_rgb
        for _ in range(numDownSamples):
            img_color = cv.pyrDown(img_color)
        
        # repeatedly apply small bilateral filters instead of one large filter
        # bilateral filter is used for smoothing and is like a Gaussian filter, except that it does not make the edges disappear
        for _ in range(numBilateralFilters):
            img_color = cv.bilateralFilter(img_color, 9, 9, 7)
        
        # upsample image to original size
        for _ in range(numDownSamples):
            img_color = cv.pyrUp(img_color)
        
        # convert to grayscale and apply median blur
        img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
        img_blur = cv.medianBlur(img_gray, 3)
        
        # detect and enhance edges
        img_edge = cv.adaptiveThreshold(img_blur, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 9, 2)
        
        # convert back to color so that it can be bit-ANDed with color image
        (x, y, z) = img_color.shape
        img_edge = cv.resize(img_edge, (y, x))
        img_edge = cv.cvtColor(img_edge, cv.COLOR_GRAY2BGR)
        cv.imwrite('edge.png', img_edge)
        
        return cv.bitwise_and(img_color, img_edge)

In [ ]:
tmp_canvas = Cartoonizer() 

# start capturing video
cap = cv.VideoCapture(0)
# capture just the first frame
ret, frame = cap.read()
# write the image to a temporary file
cv.imwrite('test.jpg', frame)
cap.release()

# use this temporary file
file_name = "test.jpg"
res = tmp_canvas.render(file_name) 
  
cv.imshow("Cartoon version", res) 
cv.waitKey(0) 
cv.destroyAllWindows() 